In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
class KNNRegression:
    def __init__ (self, X, Y, k):
        self.k = k
        self.X = X
        self.Y = Y
    
    def EuclidieanDistance (self, x1):
        if self.train == True:
            self.x2 = self.X[self.idx]
        else: 
            self.x2 = self.x_test
        distance = np.sqrt(np.sum(np.square(x1-self.x2)))
        return distance
    
    def calc_neighbors(self):
        costs = np.apply_along_axis(self.EuclidieanDistance, 1, self.X)
        sorted_costs = np.sort(costs)
        indices = np.where(costs <= sorted_costs[self.k-1])[0]
        return indices
    
    def predictions(self, indices):
        prediction = float(0)
        for i in indices:
            prediction += self.Y[i]
        prediction = prediction / indices.shape
        return float(prediction)
    
    def predict(self, x):
        self.train = False
        self.x_test = x
        indices =  self.calc_neighbors()
        print(indices)
        return self.predictions(indices)
    
    def accuracy(self, x_test_set, y_test_set):
        y_pred = []
        for i in  range(x_test_set.shape[0]):
            self.train = False
            self.x_test = x_test_set[i]
            indices =  self.calc_neighbors()
            y_pred.append(self.predictions(indices))
        ssr = np.sum((y_pred - y_test_set)**2)
        sst = np.sum((y_test_set - y_test_set.mean())**2)
        self.r2_score = 1 - (ssr/sst)
        return self.r2_score
        
    def propogate(self):
        y_pred = []
        self.train = True
        for i in range(self.X.shape[0]):
            self.idx = i
            indices =  self.calc_neighbors()
            y_pred.append(self.predictions(indices))
        print("Model Trained")

In [3]:
a = np.array([[1, 1, 1], [3, 3, 3], [2, 2, 2]])

b = np.array([1, 3, 2])
print(a.shape, b.shape)
a = KNNRegression(a, b, 2)
a.propogate()
temp_1 = np.array([[3.1, 3.1, 3.1], [2.1, 2.1, 2.1]])
temp_2 = np.array([[3.1], [2.1]])
print(temp_1.shape, temp_2.shape)
a.accuracy(np.array([[3.1, 3.1, 3.1], [2.1, 2.1, 2.1]]), np.array([[3.1], [2.1]]))

(3, 3) (3,)
Model Trained
(2, 3) (2, 1)


-1.08

In [4]:
a.predict([1, 2, 2])

[0 2]


1.5

In [5]:
# data preprocessing

df = pd.read_excel('titanic.xls')
columns_to_drop = ['name','ticket', 'home.dest']
df = df.drop(columns_to_drop, axis=1)

df['embarked'] = pd.factorize(df['embarked'])[0]

df['cabin'] = df['cabin'].fillna(df['cabin'].mode())
df['cabin'] = pd.factorize(df['cabin'])[0]

df['sex'].value_counts(dropna=False)
df['sex'] = df['sex'].map(lambda x: 1 if x == 'male' else 0)
df['sex'].value_counts()

df['fare'] = df['fare'].fillna(df['fare'].mean())

df['age'] = df['age'].fillna(df['age'].mean())

df['boat'] = df['boat'].fillna(0)
df['boat'] = pd.factorize(df['boat'])[0]

df['body'] = df['body'].fillna(df['body'].mean())

df.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked,boat,body
0,1,1,0,29.0000,0,0,211.3375,0,0,0,160.809917
1,1,1,1,0.9167,1,2,151.5500,1,0,1,160.809917
2,1,0,0,2.0000,1,2,151.5500,1,0,2,160.809917
3,1,0,1,30.0000,1,2,151.5500,1,0,2,135.000000
4,1,0,0,25.0000,1,2,151.5500,1,0,2,160.809917


In [6]:
Y = np.array(df.iloc[:, 1:2])
print(Y.shape)
X = np.array(df.iloc[:, 2:])
print(X.shape)

(1309, 1)
(1309, 9)


In [7]:
# standardization
X = X/ np.linalg.norm(X)
Y = Y.T
print(X.shape, Y.shape)

(1309, 9) (1, 1309)


In [8]:
def split_data(X, Y, test_size=0.33, random_state=42):
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = test_size, random_state = 42)
    x_train = np.array(x_train.T)
    x_test = np.array(x_test.T)
    y_train = np.array(y_train.T)
    y_test = np.array(y_test.T)
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test  = split_data(X, Y.T)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(9, 877) (9, 432) (1, 877) (1, 432)


In [9]:
model = KNNRegression(x_train, y_train.T, 5)
model.propogate()
model.accuracy(x_train, y_train.T)

Model Trained


-12.507497957584913